#### Pavlo Yatluk
#### dz_68

### ЗАВДАННЯ

#### Створіть модель згорткової нейронної мережі (CNN) із трьома шарами. Підключіть Tensorboard, та відобразіть в ньому зміну розподілу ваг моделі, обчислювальний граф моделі та зміну її метрик.

In [4]:
import torch
print(torch.__version__)

2.0.0+cpu


In [5]:
import torch
import torch.nn as nn
import torch.optim as opt
torch.set_printoptions(linewidth=120)
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.fc3 = nn.Linear(in_features=60, out_features=30)
        self.out = nn.Linear(in_features=30, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size = 2, stride = 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size = 2, stride = 2)
        x = torch.flatten(x,start_dim = 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.out(x)

        return x

In [7]:
train_set = torchvision.datasets.FashionMNIST(root="./data",
train = True,
 download=True,
transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(train_set,
                                           batch_size = 100,
                                           shuffle = True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26421880/26421880 [00:09<00:00, 2847042.72it/s]


Extracting ./data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29515/29515 [00:00<00:00, 1003517.18it/s]


Extracting ./data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4422102/4422102 [00:01<00:00, 3638678.58it/s]


Extracting ./data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5148/5148 [00:00<00:00, 5100939.52it/s]

Extracting ./data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw



In [8]:
tb = SummaryWriter()
model = CNN()
images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)
tb.add_image("images", grid)
tb.add_graph(model, images)
tb.close()

In [9]:
model = CNN()
train_loader = torch.utils.data.DataLoader(train_set,batch_size = 100, shuffle = True)
optimizer = opt.Adam(model.parameters(), lr= 0.01)
criterion = torch.nn.CrossEntropyLoss()

tb = SummaryWriter()

def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

for epoch in range(10):

    total_loss = 0
    total_correct = 0

    for images, labels in train_loader:
        images, labels = images, labels
        preds = model(images)

        loss = criterion(preds, labels)
        total_loss+= loss.item()
        total_correct+= get_num_correct(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    tb.add_scalar("Loss", total_loss, epoch)
    tb.add_scalar("Correct", total_correct, epoch)
    tb.add_scalar("Accuracy", total_correct/ len(train_set), epoch)

    tb.add_histogram("conv1.bias", model.conv1.bias, epoch)
    tb.add_histogram("conv1.weight", model.conv1.weight, epoch)
    tb.add_histogram("conv2.bias", model.conv2.bias, epoch)
    tb.add_histogram("conv2.weight", model.conv2.weight, epoch)
    tb.add_histogram("fc1.bias", model.fc1.bias, epoch)
    tb.add_histogram("fc1.weight", model.fc1.weight, epoch)
    tb.add_histogram("fc2.bias", model.fc2.bias, epoch)
    tb.add_histogram("fc2.weight", model.fc2.weight, epoch)
    tb.add_histogram("fc3.bias", model.fc3.bias, epoch)
    tb.add_histogram("fc3.weight", model.fc3.weight, epoch)

    print("epoch:", epoch, "total_correct:", total_correct, "loss:",total_loss)

tb.close()

epoch: 0 total_correct: 44338 loss: 407.5738506615162
epoch: 1 total_correct: 50746 loss: 250.64014208316803
epoch: 2 total_correct: 51588 loss: 228.87858898937702
epoch: 3 total_correct: 52088 loss: 215.01519118249416
epoch: 4 total_correct: 52337 loss: 209.67800760269165
epoch: 5 total_correct: 52556 loss: 205.91037929058075
epoch: 6 total_correct: 52738 loss: 199.2321979701519
epoch: 7 total_correct: 52989 loss: 195.7406221628189
epoch: 8 total_correct: 53114 loss: 190.28523617982864
epoch: 9 total_correct: 53180 loss: 191.32821614295244


In [ ]:
! tensorboard --logdir runs